In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm
import os

import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
df = pd.read_csv('df.csv')
df['bt']=pd.to_datetime(df['bt'])
df.set_index('bt', inplace=True)

In [3]:
def humidity_latent_event_detection(x):
    """
        Max-sum (Viterbi) algorithm 
    """
    k1 = 0.1   # true value 0.25
    k2 = 0.5    # true value 0.5

    bg_percentile = 10
    bg_window_len = 240//3

    # transition probability s1->s2 = T[s1,s2]
    T = np.array([[0.8, 0.2], 
                  [0.2, 0.8]]) 

    N = len(x)-1

    s = np.zeros((N, 2))
    s_in_x = np.zeros((N, 2))
    s_sel = np.zeros((N, 2))

    # compute the background humidity level
    x_bg = np.zeros(N+1)
    x_bg[0] = x[0]
    for n in range(N):
        if n < bg_window_len:
            idx = np.arange(0,n+1)
        else:
            idx = np.arange(n-bg_window_len+1,n+1)
        x_bg[n+1] = np.percentile(x[idx], bg_percentile)

    # prepare messages from x[n] and x[n+1] 
    for n in range(N):
        for s_ in [0, 1]:
            x_ = x[n] + (1-s_)*k1*(x_bg[n] - x[n]) + s_*k2*(100. - x[n])
            s_in_x[n, s_] = np.log(norm.pdf(x[n+1], loc=x_, scale=5))
    
    # message passing to root node (s[0]) with max-sum (Viterbi)
    s_ = np.zeros(2)
    s_in_ = np.zeros(2) # message from the future s[n]
    for n in reversed(range(N)):
        s_[0] = s_in_[0] + s_in_x[n,0]
        s_[1] = s_in_[1] + s_in_x[n,1]
        s_in_[0] = np.max([T[0,0]+s_[0], T[0,1]+s_[1]])
        s_in_[1] = np.max([T[1,0]+s_[0], T[1,1]+s_[1]])
        s_sel[n,0] = np.argmax([T[0,0]+s_[0], T[0,1]+s_[1]])
        s_sel[n,1] = np.argmax([T[1,0]+s_[0], T[1,1]+s_[1]])
        
    # decode (trace the selected path)
    s = np.zeros(N)
    s[0] = 0
    for n in range(N-1):
        s[n+1] = s_sel[n+1,int(s[n])]

    return s

In [4]:
x = df['value'].to_numpy()
s = humidity_latent_event_detection(x)
df['state'] = np.append(s,0)

In [5]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=df.index, y=df['value'], name="humidity"),
              secondary_y=False,
              )
fig.add_trace(go.Scatter(x=df.index, y=df['state'], name="state"),
              secondary_y=True,
              )
fig.add_trace(go.Scatter(x=df.index, y=df['event'], name="event"),
              secondary_y=True,
              )
fig.update_xaxes(title="date-time") # X軸タイトルを指定
fig.update_yaxes(title="humidity [%]",
                 secondary_y=False) # Y軸タイトルを指定
fig.update_yaxes(title="State",
                 secondary_y=True) # Y軸タイトルを指定
fig.update_xaxes(rangeslider={"visible":True}) # X軸に range slider を表示（下図参照）
fig.update_layout(title="Humidity") # グラフタイトルを設定

fig.write_html("hum_state_fig.html")

In [6]:
def measure_duration(x):
    """
        Measure duration x=1.0 and store the duration at the point of the rising edge
    """
    duration = np.ones(len(x)) * np.nan
    d = 0
    for n in reversed(range(len(x))):
        if x[n]:
            d += 1
        else:
            if d > 0:
                duration[n] = d
                d = 0
    return duration

In [7]:
d = measure_duration(df['state'].to_numpy())
df['state_duration'] = d

In [8]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df['state_duration']*3, mode='markers', name="state=1.0 duration"))
fig.add_trace(go.Scatter(x=df.index, y=df['duration']*3, mode='markers', name="event=1.0 duration"))
fig.update_xaxes(title="date-time") # X軸タイトルを指定
fig.update_yaxes(title="duration [min.]") # Y軸タイトルを指定
fig.update_layout(title="Humidity event duration") # グラフタイトルを設定

fig.write_html('hum_duration_fig.html')